In [1]:
import torch
from networks import FullyConnected, SPU
import numpy as np
from itertools import product
from lin_bounded_domain import LinearlyBoundedDomainVerifier
from deeppoly import DeepPolyVerifier

DEVICE = 'cpu'
INPUT_SIZE = 28


class Args:
    net: str
    spec: str

    def __init__(self, net: str, spec: str):
        self.net = net
        self.spec = spec


def analyze(net: torch.nn.Module, inputs: torch.FloatTensor, eps: float, true_label: int) -> str:
    """Returns "verified" if the returned label is the same for all points in a L-infinite epsilon-ball around an input point, and "not verified" otherwise.

    Args:
        net (torch.nn.Module): [description]
        inputs (torch.FloatTensor): [description]
        eps (float): [description]
        true_label (int): [description]

    Returns:
        [str]: Returns "verified" or "not verified"
    """
    verbose = True


    print(f"Network: {net}")


    verifier = DeepPolyVerifier(net=net, inputs=inputs,
                           eps=eps, true_label=true_label, verbose=verbose)
    
    verifier.verify()


args = Args(
    net="net1_fc1",
    spec="../test_cases/net1_fc1/example_img0_0.07500.txt"
)

with open(args.spec, 'r') as f:
    lines = [line[:-1] for line in f.readlines()]
    true_label = int(lines[0])
    pixel_values = [float(line) for line in lines[1:]]
    eps = float(args.spec[:-4].split('/')[-1].split('_')[-1])

if args.net.endswith('fc1'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [50, 10]).to(DEVICE)

elif args.net.endswith('fc2'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc3'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 10]).to(DEVICE)

elif args.net.endswith('fc4'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [100, 100, 50, 10]).to(DEVICE)

elif args.net.endswith('fc5'):
    net = FullyConnected(DEVICE, INPUT_SIZE, [
        100, 100, 100, 100, 10]).to(DEVICE)
else:
    assert False

net.load_state_dict(torch.load('../mnist_nets/%s.pt' %
                    args.net, map_location=torch.device(DEVICE)))

inputs = torch.FloatTensor(pixel_values).view(
    1, 1, INPUT_SIZE, INPUT_SIZE).to(DEVICE)


outs = net(inputs)
pred_label = outs.max(dim=1)[1].item()
assert pred_label == true_label


if analyze(net, inputs, eps, true_label):
    print('verified')
else:
    print('not verified')


Network: FullyConnected(
  (layers): Sequential(
    (0): Normalization()
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=784, out_features=50, bias=True)
    (3): SPU()
    (4): Linear(in_features=50, out_features=10, bias=True)
  )
)
Layer: Input()

Layer: Normalization()
		Weights shape: (1, 1, 28, 28)
		Bias shape: (1, 1, 28, 28)

Layer: Flatten(start_dim=1, end_dim=-1)
	Transform from Normalization()

		Weights shape: (1, 1, 28, 28, 784)
		Bias shape: (784, 1)

Layer: Linear(in_features=784, out_features=50, bias=True)
	Transform from Flatten(start_dim=1, end_dim=-1)

		Weights shape: (784, 50)
		Bias shape: (784, 1)

Layer: SPU()
	Transform from Linear(in_features=784, out_features=50, bias=True)

		Weights shape: (50, 50)
		Bias shape: (50, 1)

Layer: Linear(in_features=50, out_features=10, bias=True)
	Transform from SPU()

		Weights shape: (50, 10)
		Bias shape: (50, 1)

[Backsubstitution]

Layer 5/5: Linear(in_features=50, out_features=10, bias=True)

Pre

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 50 is different from 10)

# TESTS

In [2]:
layer_shape = (1,1,3,3)

In [5]:
[
    [
        [
            [0., 0., 0.],
            [0., 0., 0.],
            [0., 0., 0.]
        ]
    ]
]

np.zeros(layer_shape).flatten()


array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
tuple(list(map(lambda x: list(x), zip(
    *product(*map(range, layer_shape))))) * 2)


([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3],
 [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])

In [13]:
indices = tuple(list(map(lambda x: list(x), zip(*product(*map(range, layer_shape))))))

In [14]:
a = np.arange(16).reshape((4,4))
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [17]:
a[::-1]

array([[12, 13, 14, 15],
       [ 8,  9, 10, 11],
       [ 4,  5,  6,  7],
       [ 0,  1,  2,  3]])

In [3]:
a = [1,2,3,4]
a[:-1][::-1]

[3, 2, 1]

In [14]:
np.eye(4).reshape((1,1,4,4)) / 13.05

array([[[[0.07662835, 0.        , 0.        , 0.        ],
         [0.        , 0.07662835, 0.        , 0.        ],
         [0.        , 0.        , 0.07662835, 0.        ],
         [0.        , 0.        , 0.        , 0.07662835]]]])

In [8]:
a = np.ones((5, 2))
b = np.array([5, 5, 5, 5, 5])

print(f"a: {a}")
print(f"Shape of a: {a.shape}")
print(f"Shape of b: {b.shape}")
print(f"Shape of a + b: {(a + b.reshape(-1,1)).shape}")
print(f"a + b: {a + b.reshape(-1,1)}")

a: [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
Shape of a: (5, 2)
Shape of b: (5,)
Shape of a + b: (5, 2)
a + b: [[6. 6.]
 [6. 6.]
 [6. 6.]
 [6. 6.]
 [6. 6.]]


In [7]:
a = np.zeros((1,1,5, 5))
np.zeros((25,)).reshape((1,1,5,5))


array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]])

In [2]:
(51,) * 2

(51, 51)

In [7]:
np.zeros((4,4)).reshape((1,1,2,2,1,1,2,2))

array([[[[[[[[0., 0.],
             [0., 0.]]]],



          [[[[0., 0.],
             [0., 0.]]]]],




         [[[[[0., 0.],
             [0., 0.]]]],



          [[[[0., 0.],
             [0., 0.]]]]]]]])

In [5]:
shape = (1, 1, 28, 28) * 2
shape[4:]


(1, 1, 28, 28)

In [3]:
(shape[2] ** 2,)

(784,)

# `DeepPoly` test

In [ ]:
l1 = torch.nn.Linear(2, 2, device='cpu')
l1.weight = torch.Tensor([[1, 1], [1, -1]])
l1.bias = torch.Tensor([0, 0])

l2 = torch.nn.Linear(2, 2, device='cpu')
l2.weight = torch.Tensor([[1, 1], [1, -1]])
l2.bias = torch.Tensor([-0.5, 0])

l3 = torch.nn.Linear(2, 2, device='cpu')
l3.weight = torch.Tensor([[-1, 1], [0, 1]])
l3.bias = torch.Tensor([3, 0])

net = torch.nn.Sequential(l1, SPU, l2, SPU, l3, device='cpu')


input = torch.Tensor([0, 0])
eps = 1

verifier = DeepPolyVerifier(net=net, inputs=input, eps=eps, true_label=0)
verifier.verify()



In [2]:
a = np.ones((5,1))
a

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [3]:
a[:, 3]

IndexError: index 3 is out of bounds for axis 1 with size 1